In [21]:
!pip install tensorflow-gpu==1.15.0

In [22]:
# See if running on GPU
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
!pip install keras==2.2.4
!pip install matplotlib
!pip install pillow==7.1.2

In [23]:
import os
import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import cifar10, mnist
from keras.layers import (BatchNormalization, Conv2D, Conv2DTranspose, 
              Dense, Dropout, Flatten, Input, Reshape, 
              UpSampling2D, ZeroPadding2D)
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model, Sequential
from keras.optimizers import Adam

Using TensorFlow backend.


In [24]:
%matplotlib inline

In [25]:
# Consistent results
np.random.seed(10)

# The dimension of z
nosie_dim = 100

batch_size = 16
# 60000/16
ssteps_per_epoch = 3750
epochs = 21

save_path = "/content/drive/My Drive/CycleGan Monet/GANS Tutorial/fcgan-images"

im_rows, im_cols, channels = 28, 28, 1

optimizer = Adam(0.0002, 0.5)

In [26]:
if save_path != None and not os.path.isdir(save_path):
  os.mkdir(save_path)

In [27]:
# Load and Pre-process data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Normalize to between -1 and 1
x_train = (x_train.astype(np.float32) - 127.5)/127.5

x_train = x_train.reshape(-1, im_rows*im_cols*channels)
# x_train.shape == (60000, 784)

In [28]:
def create_generator():
  generator = Sequential()

  generator.add(Dense(256, input_dim=nosie_dim))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(512))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(1024))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(im_rows*im_cols*channels, activation="tanh"))

  generator.compile(loss="binary_crossentropy", optimizer=optimizer)
  return generator

In [29]:
def create_discriminator():
  discriminator = Sequential()

  discriminator.add(Dense(1024, input_dim=im_rows*im_cols*channels))
  discriminator.add(LeakyReLU(0.2))

  discriminator.add(Dense(512))
  discriminator.add(LeakyReLU(0.2))

  discriminator.add(Dense(256))
  discriminator.add(LeakyReLU(0.2))

  discriminator.add(Dense(1, activation="sigmoid"))

  discriminator.compile(loss="binary_crossentropy", optimizer=optimizer)
  return discriminator

In [30]:
# Create the generator and discriminator
discriminator = create_discriminator()
generator = create_generator()






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [31]:
# Make the discriminator untrainable when we are training the 
# generator. This does not effect the discriminator by itself
discriminator.trainable=False

# Link the two models to create the GAN
gan_input = Input(shape=(nosie_dim,))
fake_image = generator(gan_input)

gan_output = discriminator(fake_image)

gan = Model(gan_input, gan_output)
gan.compile(loss="binary_crossentropy", optimizer=optimizer)

In [32]:
# Display images and save them if the epoch number is specified
def show_image(noise, epoch=None):
  generated_images = generator.predict(noise)
  plt.figure(figsize=(10, 10))

  for i, image in enumerate(generated_images):
    plt.subplot(10, 10, i+1)
    if channels == 1:
      plt.imshow(image.reshape((im_rows, im_cols)), cmap="gray")
    else:
      plt.imshow(image.reshape((im_rows, im_cols, channels)))
    plt.axis("off")

  plt.tight_layout()

  if epoch != None and save_path != None:
    plt.savefig("{}/gan-images_epoch-{}.png".format(save_path, epoch))

In [33]:
# Constant Noise for viewing how the GAN process (Generate pseudo 
# image from these noise)
static_noise = np.random.normal(0, 1, size=(100, nosie_dim))

# train the GAN
for epoch in range(epochs):
  for batch in range(ssteps_per_epoch):
    noise = np.random.normal(0, 1, size=(batch_size, nosie_dim))
    fake_x = generator.predict(noise)

    real_x = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]

    # Concatenate the data with label 1s and 0s
    x = np.concatenate((real_x, fake_x))

    disc_y = np.zeros(2*batch_size)
    # label smoothing
    disc_y[:batch_size] = 0.9

    d_loss = discriminator.train_on_batch(x, disc_y)

    y_gen = np.ones(batch_size)
    g_loss = gan.train_on_batch(noise, y_gen)

  print("Epoch: {} \t Discriminator Loss: {} \t\t Generator Loss: {}".format(epoch, d_loss, g_loss))
  if epoch % 2 == 0:
    show_image(static_noise, epoch)

Output hidden; open in https://colab.research.google.com to view.

In [35]:
# Turn the training process into GIF
from PIL import Image, ImageDraw

frames = []
for image in os.listdir(save_path):
  frames.append(Image.open(save_path + "/" + image))
frames[0].save("/content/drive/My Drive/CycleGan Monet/GANS Tutorial/fcgan_taining.gif", format="GIF", append_images=frames[1:], save_all=True, duration=500, loop=0)

In [36]:
# Save the model
discriminator.save("/content/drive/My Drive/CycleGan Monet/GANS Tutorial/fcdiscriminator.hdf5")
generator.save("/content/drive/My Drive/CycleGan Monet/GANS Tutorial/fcgenerator.hdf5")
gan.save("/content/drive/My Drive/CycleGan Monet/GANS Tutorial/fcgan.hdf5")

## Notes  
* Why Activation Function: https://towardsdatascience.com/everything-you-need-to-know-about-activation-functions-in-deep-learning-models-84ba9f82c253  
* Leaky Relu: https://ml-cheatsheet.readthedocs.io/en/latest/activation_functions.html#leakyrelu  
* Mode Collapse of GANS: https://aiden.nibali.org/blog/2017-01-18-mode-collapse-gans/  
* One-sided label smoothing: https://arxiv.org/pdf/1606.03498.pdf

In [40]:
!ls

'GANS Tutorial'   README.md


In [39]:
cd /content/drive/My\ Drive/CycleGan\ Monet

/content/drive/My Drive/CycleGan Monet


In [41]:
!git init 

Initialized empty Git repository in /content/drive/My Drive/CycleGan Monet/.git/


In [42]:
!git config --global user.email “mingzhehu511@gmail.com”
!git config --global user.name “MingzheHu-Duke”

In [28]:
!rm -f ./.git/index.lock

In [65]:
!git add .

In [66]:
!git commit -m "Removed personal password"

[master 4b855a9] Removed personal password
 1 file changed, 1 insertion(+), 1 deletion(-)


In [62]:
!git remote rm origin

In [63]:
!git remote add origin https://MingzheHu-Duke:<password>@github.com/MingzheHu-Duke/CycleGAN.git

In [64]:
!git push -u origin master

Counting objects: 22, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (21/21), done.
Writing objects: 100% (22/22), 30.18 MiB | 6.51 MiB/s, done.
Total 22 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), done.
To https://github.com/MingzheHu-Duke/CycleGAN.git
 * [new branch]      master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.


In [67]:
!git push

Counting objects: 4, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.37 KiB | 279.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/MingzheHu-Duke/CycleGAN.git
   3702a11..4b855a9  master -> master
